In [1]:
'''
import logging
import sys
my_stderr = sys.stderr = open('errors.txt', 'w')  # redirect stderr to file
get_ipython().log.handlers[0].stream = my_stderr  # log errors to new stderr
get_ipython().log.setLevel(logging.INFO)  # errors are logged at info level
'''

"\nimport logging\nimport sys\nmy_stderr = sys.stderr = open('errors.txt', 'w')  # redirect stderr to file\nget_ipython().log.handlers[0].stream = my_stderr  # log errors to new stderr\nget_ipython().log.setLevel(logging.INFO)  # errors are logged at info level\n"

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType
from pyspark.sql import functions as F

In [3]:
from pyspark import SparkConf,SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import Row,SQLContext
import requests
import sys

In [4]:
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/nikita/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [5]:
def compound_detection(text):
    sia = SentimentIntensityAnalyzer()
    return sia.polarity_scores(text)["compound"]

def pos_detection(text):
    sia = SentimentIntensityAnalyzer()
    return sia.polarity_scores(text)["pos"]

def neu_detection(text):
    sia = SentimentIntensityAnalyzer()
    return sia.polarity_scores(text)["neu"]

def neg_detection(text):
    sia = SentimentIntensityAnalyzer()
    return sia.polarity_scores(text)["neg"]

def sentiment_analysis(tweets):
    compound_detection_udf = F.udf(compound_detection, StringType())
    tweets = tweets.withColumn("comp", compound_detection_udf("tweet"))
    
    positivity_detection_udf = F.udf(pos_detection, StringType())
    tweets = tweets.withColumn("pos", positivity_detection_udf("tweet"))
    
    neu_detection_udf = F.udf(neu_detection, StringType())
    tweets = tweets.withColumn("neu", neu_detection_udf("tweet"))
    
    neg_detection_udf = F.udf(neg_detection, StringType())
    tweets = tweets.withColumn("neg", neg_detection_udf("tweet"))
    
    return tweets

In [6]:
def clean_tweets(tweets):
    tweets = tweets.na.replace('', None)
    tweets = tweets.na.drop()

    # Clean Tweet Text 
    tweets = tweets.withColumn('tweet', F.regexp_replace('tweet', r'http\S+', ''))
    tweets = tweets.withColumn('tweet', F.regexp_replace('tweet', r'(#\w+)', ''))
    tweets = tweets.withColumn('tweet', F.regexp_replace('tweet', r'(@\w+)', ''))
    tweets = tweets.withColumn('tweet', F.regexp_replace('tweet', ':', ''))
    tweets = tweets.withColumn('tweet', F.regexp_replace('tweet', 'RT', ''))
    tweets = tweets.withColumn('tweet', F.regexp_replace('tweet', r'[^a-zA-Z0-9 -]', ''))
    tweets = tweets.withColumn('tweet', F.trim(tweets.tweet))
    tweets.show(2, False)
    return tweets

In [7]:
def aggregate_data_count(new_values, total_sum):
    return sum(new_values) + (total_sum or 0)

In [8]:
def get_sql_context_instance(spark_context):
    if ('sqlContextSingletonInstance' not in globals()):
        globals()['sqlContextSingletonInstance'] = SQLContext(spark_context)
    return globals()['sqlContextSingletonInstance']

In [9]:
def send_hashtag_df_to_dashboard(df):
    # extract the hashtags from dataframe and convert them into arraycompound
    top_tags = [str(t.hashtag) for t in df.select("hashtag").collect()]
    # extract the counts from dataframe and convert them into array
    tags_count = [p.hashtag_count for p in df.select("hashtag_count").collect()]
    # initialize and send the data through REST API
    url = 'http://localhost:5001/hashtags/updateData'
    request_data = {'label': str(top_tags), 'data': str(tags_count)}
    response = requests.post(url, data=request_data)

In [10]:
def send_user_mentions_df_to_dashboard(df):
    # extract the mentions from dataframe and convert them into array
    top_users = [str(t.mention) for t in df.select("mention").collect()]
    # extract the counts from dataframe and convert them into array
    users_count = [p.mention_count for p in df.select("mention_count").collect()]
    # initialize and send the data through REST API
    url = 'http://localhost:5001/mentions/updateData'
    request_data = {'label': str(top_users), 'data': str(users_count)}
    response = requests.post(url, data=request_data)

In [11]:
def send_sentiment_analysis_to_dashboard(df):
    # extract sentiment analysis results
    comp_lst = [float(t.comp) for t in df.select("comp").collect()]
    comp_mean = sum(comp_lst)/len(comp_lst)
    pos_lst = [float(t.pos) for t in df.select("pos").collect()]
    pos_mean = sum(pos_lst)/len(pos_lst)
    neu_lst = [float(t.neu) for t in df.select("neu").collect()]
    neu_mean = sum(neu_lst)/len(neu_lst)
    neg_lst = [float(t.neg) for t in df.select("neg").collect()]
    neg_mean = sum(neg_lst)/len(neg_lst)
    url = 'http://localhost:5001/sentiment/updateData'
    request_data = {'comp': str(comp_mean), 'pos': str(pos_mean), 'neu': str(neu_mean), 'neg': str(neg_mean)}
    response = requests.post(url, data=request_data)

In [12]:
def process_rdd_hashtags(time, rdd):
    print("HASHTAG----------- %s -----------" % str(time))
    try:
        # Get spark sql singleton context from the current context
        sql_context = get_sql_context_instance(rdd.context)
        # convert the RDD to Row RDD
        row_rdd = rdd.map(lambda w: Row(hashtag=w[0], hashtag_count=w[1]))
        # create a DF from the Row RDD
        hashtags_df = sql_context.createDataFrame(row_rdd)
        # Register the dataframe as table
        hashtags_df.registerTempTable("hashtags")
        # get the top 10 hashtags from the table using SQL and print them
        hashtag_counts_df = sql_context.sql("select hashtag, hashtag_count from hashtags order by hashtag_count desc limit 20")
        # call this method to prepare top 10 hashtags DF and send them
        hashtag_counts_df.show(2)
        send_hashtag_df_to_dashboard(hashtag_counts_df)
    except:
        e = sys.exc_info()[0]
        print("Error: %s" % e)

In [13]:
def process_rdd_mentions(time, rdd):
    print("MENTION----------- %s -----------" % str(time))
    try:
        # Get spark sql singleton context from the current context
        sql_context = get_sql_context_instance(rdd.context)
        # convert the RDD to Row RDD
        row_rdd = rdd.map(lambda w: Row(mention=w[0], mention_count=w[1]))
        # create a DF from the Row RDD
        mentions_df = sql_context.createDataFrame(row_rdd)
        # Register the dataframe as table
        mentions_df.registerTempTable("mentions")
        # get the top 10 hashtags from the table using SQL and print them
        mentions_count_df = sql_context.sql("select mention, mention_count from mentions order by mention_count desc limit 20")
        # call this method to prepare top 10 hashtags DF and send them
        send_user_mentions_df_to_dashboard(mentions_count_df)
    except:
        e = sys.exc_info()[0]
        print("Error: %s" % e)

In [14]:
def process_rdd_sentiment(time, rdd):
    print("SENTIMENT----------- %s -----------" % str(time))
    try:
        # Get spark sql singleton context from the current context
        sql_context = get_sql_context_instance(rdd.context)
        # convert the RDD to Row RDD
        row_rdd = rdd.map(lambda w: Row(tweet=w[0], comp=w[1]))
        # create a DF from the Row RDD
        sentiment_df = sql_context.createDataFrame(row_rdd)
        # Register the dataframe as table
        sentiment_df.registerTempTable("sentiment")
        # get the top 10 hashtags from the table using SQL and print them
        sentiment_raw_df = sql_context.sql("select tweet, comp from sentiment order by comp desc limit 50")
        sentiment_clean_df = clean_tweets(sentiment_raw_df)
        sentiment_analyzed_df = sentiment_analysis(sentiment_clean_df)
        # call this method to prepare comp mean DF and send it
        send_sentiment_analysis_to_dashboard(sentiment_analyzed_df)
    except:
        e = sys.exc_info()[0]
        print("Error: %s" % e)

In [15]:
# create spark configuration
conf = SparkConf()
conf.setAppName("TwitterStreamApp")
# create spark context with the above configuration
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")
# create the Streaming Context from the above spark context with interval size 2 seconds
ssc = StreamingContext(sc, 2)
# setting a checkpoint to allow RDD recovery
ssc.checkpoint("checkpoint_TwitterApp")
# read data from port 9009
dataStream = ssc.socketTextStream("0.0.0.0",5555)

21/07/29 23:09:25 WARN Utils: Your hostname, nikita-Blade resolves to a loopback address: 127.0.1.1; using 192.168.99.126 instead (on interface wlp2s0)
21/07/29 23:09:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/07/29 23:09:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [16]:
# split each line into tweets
tweets = dataStream.flatMap(lambda line: line.split("t_end"))
# split each tweet into words
words = dataStream.flatMap(lambda text: text.split(" "))

In [17]:
# filter the words to get only hashtags/mentions, then map each hashtag/mention to be a pair of (hashtag/mention,1)
hashtags = words.filter(lambda w: '#' in w).map(lambda x: (x.upper(), 1))
mentions = words.filter(lambda w: '@' in w).map(lambda x: (x.upper(), 1))
sentiment = tweets.filter(lambda w: ' ' in w).map(lambda x: (x, 0.0))

In [18]:
# adding the count of each hashtag/mention to its last countsentiment_values_df
hashtags_totals = hashtags.updateStateByKey(aggregate_data_count)
mentions_totals = mentions.updateStateByKey(aggregate_data_count)
sentiment_totals = sentiment.updateStateByKey(aggregate_data_count)

In [19]:
# do processing for each RDD generated in each interval
hashtags_totals.foreachRDD(process_rdd_hashtags)
mentions_totals.foreachRDD(process_rdd_mentions)
sentiment_totals.foreachRDD(process_rdd_sentiment)

In [ ]:
try:
    # start the streaming computation
    ssc.start()
    # wait for the streaming to finish
    ssc.awaitTermination()
except KeyboardInterrupt:
    ssc.stop()
    print('Interrupted')

HASHTAG----------- 2021-07-29 23:09:28 -----------


Error: <class 'ValueError'>
MENTION----------- 2021-07-29 23:09:28 -----------
Error: <class 'ValueError'>
SENTIMENT----------- 2021-07-29 23:09:28 -----------
Error: <class 'ValueError'>
HASHTAG----------- 2021-07-29 23:09:30 -----------


+---------+-------------+
|  hashtag|hashtag_count|
+---------+-------------+
|   #COVID|           16|
|#COVID-19|            4|
+---------+-------------+
only showing top 2 rows

MENTION----------- 2021-07-29 23:09:30 -----------
SENTIMENT----------- 2021-07-29 23:09:30 -----------


+--------------------------------------------------------------------------------------------------------------------+----+
|tweet                                                                                                               |comp|
+--------------------------------------------------------------------------------------------------------------------+----+
|The founder of Shake Shack says his company will require proof of Covid vaccination for both employees AND customers|0.0 |
|Figure that COVID isnt trying to destroy a political party it is trying to destroy an entire                        |0.0 |
+--------------------------------------------------------------------------------------------------------------------+----+
only showing top 2 rows



HASHTAG----------- 2021-07-29 23:09:32 -----------
+--------+-------------+
| hashtag|hashtag_count|
+--------+-------------+
|  #COVID|           31|
|#COVID19|            8|
+--------+-------------+
only showing top 2 rows

MENTION----------- 2021-07-29 23:09:32 -----------
SENTIMENT----------- 2021-07-29 23:09:32 -----------
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+
|tweet                                                                                                                                                                                                                                                         |comp|
+-------------------------------------------------------------------------------------------------------------------------------------------------

...

Error: <class 'requests.exceptions.ConnectionError'>
HASHTAG----------- 2021-07-29 23:11:38 -----------
+--------+-------------+
| hashtag|hashtag_count|
+--------+-------------+
|  #COVID|          315|
|#COVID19|           76|
+--------+-------------+
only showing top 2 rows

Error: <class 'requests.exceptions.ConnectionError'>
MENTION----------- 2021-07-29 23:11:38 -----------
Error: <class 'requests.exceptions.ConnectionError'>
SENTIMENT----------- 2021-07-29 23:11:38 -----------
+--------------------------------------------------------------------------------------------------------------------+----+
|tweet                                                                                                               |comp|
+--------------------------------------------------------------------------------------------------------------------+----+
|The founder of Shake Shack says his company will require proof of Covid vaccination for both employees AND customers|0   |
|Figure that CO

Error: <class 'requests.exceptions.ConnectionError'>
HASHTAG----------- 2021-07-29 23:11:44 -----------
+--------+-------------+
| hashtag|hashtag_count|
+--------+-------------+
|  #COVID|          315|
|#COVID19|           76|
+--------+-------------+
only showing top 2 rows

Error: <class 'requests.exceptions.ConnectionError'>
MENTION----------- 2021-07-29 23:11:44 -----------
Error: <class 'requests.exceptions.ConnectionError'>
SENTIMENT----------- 2021-07-29 23:11:44 -----------
+--------------------------------------------------------------------------------------------------------------------+----+
|tweet                                                                                                               |comp|
+--------------------------------------------------------------------------------------------------------------------+----+
|The founder of Shake Shack says his company will require proof of Covid vaccination for both employees AND customers|0   |
|Figure that CO

Error: <class 'requests.exceptions.ConnectionError'>
HASHTAG----------- 2021-07-29 23:11:46 -----------
+--------+-------------+
| hashtag|hashtag_count|
+--------+-------------+
|  #COVID|          315|
|#COVID19|           76|
+--------+-------------+
only showing top 2 rows

Error: <class 'requests.exceptions.ConnectionError'>
MENTION----------- 2021-07-29 23:11:46 -----------
Error: <class 'requests.exceptions.ConnectionError'>
SENTIMENT----------- 2021-07-29 23:11:46 -----------
+--------------------------------------------------------------------------------------------------------------------+----+
|tweet                                                                                                               |comp|
+--------------------------------------------------------------------------------------------------------------------+----+
|The founder of Shake Shack says his company will require proof of Covid vaccination for both employees AND customers|0   |
|Figure that CO

Error: <class 'requests.exceptions.ConnectionError'>
HASHTAG----------- 2021-07-29 23:11:48 -----------
+--------+-------------+
| hashtag|hashtag_count|
+--------+-------------+
|  #COVID|          315|
|#COVID19|           76|
+--------+-------------+
only showing top 2 rows

Error: <class 'requests.exceptions.ConnectionError'>
MENTION----------- 2021-07-29 23:11:48 -----------
Error: <class 'requests.exceptions.ConnectionError'>
SENTIMENT----------- 2021-07-29 23:11:48 -----------
+--------------------------------------------------------------------------------------------------------------------+----+
|tweet                                                                                                               |comp|
+--------------------------------------------------------------------------------------------------------------------+----+
|The founder of Shake Shack says his company will require proof of Covid vaccination for both employees AND customers|0   |
|Figure that CO

Error: <class 'requests.exceptions.ConnectionError'>
HASHTAG----------- 2021-07-29 23:11:50 -----------
+--------+-------------+
| hashtag|hashtag_count|
+--------+-------------+
|  #COVID|          315|
|#COVID19|           76|
+--------+-------------+
only showing top 2 rows

Error: <class 'requests.exceptions.ConnectionError'>
MENTION----------- 2021-07-29 23:11:50 -----------
Error: <class 'requests.exceptions.ConnectionError'>
SENTIMENT----------- 2021-07-29 23:11:50 -----------
+--------------------------------------------------------------------------------------------------------------------+----+
|tweet                                                                                                               |comp|
+--------------------------------------------------------------------------------------------------------------------+----+
|The founder of Shake Shack says his company will require proof of Covid vaccination for both employees AND customers|0   |
|Figure that CO